In [9]:
import nighres
import nibabel as nb
import numpy as np
from nipype.interfaces import fsl

### Make levelsets

In [3]:
gmlvl = nighres.surface.probability_to_levelset("/home/julia/data/gradients/atlas/allen_api/cortex_mask_tight.nii.gz",
                                                save_data=True)


Probability to Levelset

Outputs will be saved to /home/julia/data/gradients/atlas/allen_api/

 The underlying Java code did not execute cleanly: 
<class 'nighresjava.JavaError'>


JavaError: <super: <class 'JavaError'>, <JavaError object>>
    Java stacktrace:
java.lang.OutOfMemoryError: Java heap space
	at de.mpg.cbs.methods.InflateGdm$NarrowBand.<init>(InflateGdm.java:110)
	at de.mpg.cbs.methods.InflateGdm.evolveNarrowBand(InflateGdm.java:494)
	at de.mpg.cbs.core.surface.SurfaceProbabilityToLevelset.execute(SurfaceProbabilityToLevelset.java:98)


In [ ]:
# inner.nii.gz is the masks for layer6a and layer6b together
wmlvl = nighres.surface.probability_to_levelset("/home/julia/data/gradients/atlas/allen_api/profiles/inner_mask.nii.gz",
                                                save_data=True)


Probability to Levelset

Outputs will be saved to /home/julia/data/gradients/atlas/allen_api/layers/


### Layering

https://github.com/juhuntenburg/nighres/tree/allen_brain

Volumetric layering with following changes in the code:
* algo.setLayeringMethod("distance-preserving")
* algo.setCurvatureApproximationScale(5)
* algo.setPresmoothCorticalSurfaces(True)

In [ ]:
# Ran this on the cluster
lamina = nighres.laminar.volumetric_layering(inner_levelset=wmlvl['result'],
                                             outer_levelset=gmlvl['result'],
                                             n_layers=5,
                                             save_data=True,
                                             overwrite=True)
# Renamed outputs to profiles_lvl.nii.gz, profiles_depth.nii.gz, profiles_layers.nii.gz

### Downsample levelset

In [10]:
resamp = fsl.FLIRT(in_file="/home/julia/data/gradients/atlas/allen_api/profiles/profiles_lvl.nii.gz",
                   reference="/home/julia/data/gradients/atlas/allen_api/layers/profiles_lvl.nii.gz",
                   apply_isoxfm=0.2,
                   interp="spline",
                   out_file="/home/julia/data/gradients/atlas/allen_api/layers/profiles_lvl_200um.nii.gz")
resamp.run()

191209-17:27:58,392 interface INFO:
	 stderr 2019-12-09T17:27:58.391597:Image Exception : #99 :: Out of memory
191209-17:27:58,630 interface INFO:
	 stderr 2019-12-09T17:27:58.630819:Out of memory


TraitError: The trait 'out_file' of a FLIRTOutputSpec instance is an existing file name, but the path  '/home/julia/data/gradients/atlas/allen_api/layers/inner_p2l-surf_layering-boundaries_200um.nii.gz' does not exist.

### Sample annotation in high resolution space

In [8]:
# Ran this on the cluster
nighres.laminar.profile_sampling(profile_surface_image='profiles_lvl.nii.gz',
                                 intensity_image='annotation.nii.gz',
                                 save_data=True, overwrite=True)
# Rename output to annotation_profile_sampling.nii.gz 
# Split into annotation_profile_0.nii.gz to annotation_profile_5.nii.gz